
#22. Top 5 de lenguajes que son usados por usuarios bilingües. (⭐⭐).


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=fddab3c787ff6f3385fc4e92d7b1e100c21fc50744de5306f28d4b659d781277
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/drive/MyDrive/Datasets/languages.csv')

In [ ]:
languages_rdd = df.rdd

In [ ]:
languages_rdd

MapPartitionsRDD[14] at javaToPython at NativeMethodAccessorImpl.java:0

In [ ]:
languages_rdd.getNumPartitions()

1

In [ ]:
languages_rdd.take(5)

[Row(babel_user=4502458, babel_lang='FR', babel_level='2'),
 Row(babel_user=5928200, babel_lang='aa', babel_level='0'),
 Row(babel_user=46918, babel_lang='ab', babel_level='0'),
 Row(babel_user=2050449, babel_lang='ab', babel_level='1'),
 Row(babel_user=4715583, babel_lang='ace', babel_level='0')]

Armo el RDD para que tenga usuario e idioma que utiliza (en minuscula asi no hay diferencias), después agrupé por usuario para conocer todos los idiomas que habla, y si habla más de 1 idioma lo considero bilingue. El segundo map es para poder ver los idiomas en una lista.

In [ ]:
bilingues_rdd = languages_rdd.map(lambda x: (x[0], x[1].lower())).groupByKey().filter(lambda x: len(x[1]) > 1)

Ahora desarmo ese RDD y me quedo con un RDD que contenga (idioma, 1) para poder sumar cada idioma y ver cuales son los 5 más utilizados

In [ ]:
resultado = bilingues_rdd.flatMap(lambda x: x[1]).map(lambda x: (x, 1)).reduceByKey(lambda a, b : a+b).takeOrdered(5, lambda x: -x[1])

In [ ]:
resultado

[('es', 9439), ('en', 9345), ('fr', 3545), ('it', 1786), ('de', 1727)]